### First-Come-First-Served 

Implementation of simple FCFS algorithm based on Travis's framework. <br>
If the new schedule can be asigned in original schedule, it keeps the original schedules. 

In [93]:
# import the necessary packages
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# load the data
df = pd.read_csv("./data/21DEC2023_AMS_processed.csv", parse_dates=['time_sch', 'time_act'])
df.tail(5)

,time_sch,time_act,code,dest,stat,orig,pass_load,time_diff
1195,2023-12-21 23:25:00,2023-12-22 00:21:00,KL 1608 KLM,Amsterdam,BAGGAGE ON BELT,Rome (FCO),309,-83040.0
1196,2023-12-21 23:30:00,2023-12-22 00:29:00,HV 5806 Transavia,Amsterdam,BAGGAGE ON BELT,Thessaloniki (SKG),310,-82860.0
1197,2023-12-21 23:40:00,2023-12-22 00:27:00,HV 6902 Transavia,Amsterdam,BAGGAGE ON BELT,Dubai International (DXB),316,-83580.0
1198,2023-12-21 23:50:00,2023-12-22 00:52:00,HV 5666 Transavia,Amsterdam,BAGGAGE ON BELT,Las Palmas de Gran Canaria (LPA),347,-82680.0
1199,2023-12-21 23:55:00,2023-12-22 01:00:00,HV 5218 Transavia,Amsterdam,BAGGAGE ON BELT,Catania (CTA),304,-82500.0


In [94]:
class reschedule_problem:

    def __init__(self,df, n_runway = 1, resume_hour = 23, resume_min = 0,\
                timeslot_dur = 5, max_delay = 120):
        # customized attributes of the problem
        self.df = df
        self.move = df['code']
        self.n_runway = n_runway
        self.hour = resume_hour 
        self.min = resume_min 
        self.time_slot_duration = timeslot_dur
        self.max_delay = max_delay
        # parsing the attribute for more useful format
        self.date = self.df.loc[0,'time_sch'].day
        self.month = self.df.loc[0,'time_sch'].month
        self.year = self.df.loc[0,'time_sch'].year
        self.initial = pd.DataFrame(columns = ["time_new","util"], index= []) # to store the solution

    def actions(self, state):
        """Return the next batch of flights to land and are not assigned a time slot."""
        # Get the list of flights that haven't been assigned a new time slot yet
        assigned_flights = list(state.index)
        unassigned_flights = self.df[~self.df['code'].isin(assigned_flights)].copy()

        # Sort the unassigned flights by their original schedule
        unassigned_flights_sorted = unassigned_flights.sort_values(by='time_sch')

        # Select the next batch of flights based on the number of available runways
        next_flights = unassigned_flights_sorted.head(self.n_runway)['code'].tolist()
        return next_flights, []
        
        
    def parse_state_time(self,state):
        """Use to get the value in the schedule and 
        parse the time of the next time slot"""
        if len(state) <=0: 
            year = self.year
            month = self.month
            date = self.date
            hour = self.hour
            min = self.min

        else:
            timestr = state['time_new'].iloc[-1] # get the time_sch
            year, month, date, time = timestr.split(" ")
            hour , min = time.split(":")
            # get the current time
            min = int(min)
            min += self.time_slot_duration
            if min // 60 <= 1:
                hour = int(hour) + min // 60
                min = min % 60
                if hour // 24 <= 1:
                    date = int(date) + hour // 24
                    hour = hour % 24
                
        return year, month, date, hour,min
    
    def result(self, state, flight_diverted, flights):
        """Return the state that is the result of scheduling the given flights."""
        year, month, date, hour, min = self.parse_state_time(state)
        ori_min = min
        # Count the number of flights already scheduled in this time slot
        flights_in_slot = sum(state['time_new'] == f"{year} {month} {date} {hour:02d}:{min:02d}")
        for fl in flights:
            time_sch_org = self.df.query("code == @fl")['time_sch'].iloc[0] # type pd Series
            time_sch_new = pd.to_datetime(f"{year} {month} {date} {hour}:{min}")
            if (time_sch_org - time_sch_new).total_seconds() < 0:
                time_sch = f"{year} {month} {date} {hour:02d}:{min:02d}"
                util = self.compute_util(fl, year, month, date, hour, min)
            else: 
                time_sch = time_sch_org.strftime("%Y %m %d %H:%M")
                util = 0
            state.loc[fl] = [time_sch, util]

            # Increment flight count and time slot only if the slot is full
            flights_in_slot += 1
            if flights_in_slot >= self.n_runway:
                min = ori_min + self.time_slot_duration
                flights_in_slot = 0
                if min >= 60:
                    hour += 1
                    min -= 60
                    if hour >= 24:
                        date += 1
                        hour -= 24
        return state.sort_values("time_new")
    
    def compute_util(self, flcode, year, month, date, hour, min):
        """Compute the utility of a given rescheduled flight"""
        if flcode is None:
            return 0 
        elif year != 1970:
            time_sch_org = self.df.query("code == @flcode")['time_sch'] # type pd Series
            time_sch_new = pd.to_datetime(f"{year} {month} {date} {hour}:{min}")
            # compute the time delayed
            delay = time_sch_org - time_sch_new
            delay = delay.reset_index(drop = True)[0].total_seconds() / 60
            return delay
        elif year == 1970:
            # compute utility of diverted flight
            delay = - self.max_delay 
            return delay
    
    def utility(self, state):
        """Compute aggregate utility of a given state"""
        agg_util = state['util'].sum()
        return agg_util
        
    def goal_test(self, state):
        "return True if the state is terminal"
        flight_assigned = [flight for flight in state.index] 
        if len(flight_assigned) == len(self.df):
        # !!! this is not a robust way since time slot can be none
            return True
        
    def display(self):
        """ Method to show the new schedule of the flightafter solving for the problem, 
        """
        display_df = self.df.copy()
        display_df = pd.merge(self.df[['code','time_sch']], display_df, left_on= "code", right_index = True)
        display_df['time_dff'] = display_df.apply(lambda x: (x['time_sch'] - x['time_new']).total_seconds()/ 60,axis = 1 )
        display_df['time_dff'] = display_df['time_dff'].apply(lambda x: -x if x <= 0 else "diverted" )
        return display_df

In [95]:
df_subset = df.tail(20).reset_index(drop = True)
resumption = df_subset.iloc[0,0] + pd.Timedelta(minutes = 30)

problem = reschedule_problem(df = df_subset, n_runway=2,
                                   resume_hour= resumption.hour, resume_min=resumption.minute,
                                   max_delay = 25)
# Run the scheduling algorithm
while not problem.goal_test(problem.initial):
    flights_to_schedule, flights_diverted = problem.actions(problem.initial)
    problem.initial = problem.result(problem.initial, flights_diverted, flights_to_schedule)

# Display the result
display_df = pd.merge(problem.initial, problem.df[["code", 'time_sch']], left_index=True, right_on="code")
display_df = display_df.set_index(["code"])
display_df[['time_sch', "time_new", "util"]]

,time_sch,time_new,util
code,,,
OR 3802 TUI fly,2023-12-21 22:30:00,2023 12 21 23:00,-30.0
LH 2310 Lufthansa,2023-12-21 22:35:00,2023 12 21 23:00,-25.0
KL 1032 KLM,2023-12-21 22:35:00,2023 12 21 23:05,-30.0
AZ 118 ITA Airways,2023-12-21 22:40:00,2023 12 21 23:05,-25.0
KL 1118 KLM,2023-12-21 23:00:00,2023 12 21 23:10,-10.0
KL 1434 KLM,2023-12-21 23:00:00,2023 12 21 23:10,-10.0
KL 1706 KLM,2023-12-21 23:00:00,2023 12 21 23:15,-15.0
KL 980 KLM,2023-12-21 23:00:00,2023 12 21 23:15,-15.0
KL 1136 KLM,2023-12-21 23:05:00,2023 12 21 23:20,-15.0
